In [1]:
import pandas as pd

In [2]:
import importlib

In [3]:
import geopandas as gp

In [4]:
from shapely.geometry import Point

In [5]:
vrp = importlib.import_module("VRP_OR-tools_Stops_veh_tech")

In [6]:
import pickle

In [7]:
from unittest import TestCase

In [8]:
# Testing normal case where the origin and destination exist in the origin-destination travel time dataframe
def test_tt_cal_normal():
    sel_tt = pd.DataFrame()
    sel_tt['origin'] = [1000, 1000,1001,1001]
    sel_tt['destination'] = [1000,1001,1000,1001]
    sel_tt['TIME_minutes'] = [0,5,10,0]
    
    sel_dist = pd.DataFrame()
    sel_dist['Origin'] = [10, 10,12,12]
    sel_dist['Destination'] = [10,12,10,12]
    sel_dist['dist'] = [0,500,1000,0]
    
    assert vrp.tt_cal(10, 10, 1000, 1001, sel_tt, sel_dist) == 5, "incorrect travel time"

In [9]:
# Testing case where the origin and destination are not in the travel time dataframe 
# but in the orgin-destination distance dataframe
def test_tt_cal_dist():
    sel_tt = pd.DataFrame()
    sel_tt['origin'] = [1000, 1000,1001,1001]
    sel_tt['destination'] = [1000,1001,1000,1001]
    sel_tt['TIME_minutes'] = [0,5,10,0]
    
    sel_dist = pd.DataFrame()
    sel_dist['Origin'] = [10, 10,12,12]
    sel_dist['Destination'] = [10,12,10,12]
    sel_dist['dist'] = [0,500,1000,0]
    
    assert vrp.tt_cal(10, 12, 1005, 1012, sel_tt, sel_dist) == 750, "incorrect travel time"

In [10]:
# Testing case where the origin and destination are not in the travel time dataframe 
# and are not in the orgin-destination distance dataframe
def test_tt_cal_exception():
    sel_tt = pd.DataFrame()
    sel_tt['origin'] = [1000, 1000,1001,1001]
    sel_tt['destination'] = [1000,1001,1000,1001]
    sel_tt['TIME_minutes'] = [0,5,10,0]
    
    sel_dist = pd.DataFrame()
    sel_dist['Origin'] = [10, 10,12,12]
    sel_dist['Destination'] = [10,12,10,12]
    sel_dist['dist'] = [0,500,1000,0]
    
    assert vrp.tt_cal(13, 15, 1005, 1012, sel_tt, sel_dist) == 180, "incorrect travel time"

In [11]:
CBGzone_df = gp.read_file('../../../../FRISM_input_output_AT/Sim_inputs/Geo_data/Austin_freight_centroids.geojson')

In [25]:
CBGzone_df.head()

,GEOID,CBPZONE,MESOZONE,geometry
0,483030018015,48303,4058.0,POINT (-101.91808 33.55102)
1,480291812003,48029,1188.0,POINT (-98.53425 29.53896)
2,481279502001,48127,2439.0,POINT (-99.97708 28.47273)
3,480291906041,48029,1367.0,POINT (-98.51800 29.47797)
4,480291715023,48029,1032.0,POINT (-98.57718 29.42446)


In [28]:
CBGzone_df.dtypes

GEOID         object
CBPZONE        int64
MESOZONE     float64
geometry    geometry
dtype: object

In [36]:
def test_get_geoId_normal():
    """ Testing getting geo id (census block Id) from geo Id files, case where the geo id exist in the file 
    """
    pd_df = pd.DataFrame()
    pd_df['GEOID'] = [456, 457]
    pd_df['CBPZONE'] = [1234, 1235]
    pd_df['MESOZONE'] = [101,1001]
    pd_df['geometry'] = [Point(-101.91, 33.55), Point(-98.53, 29.53)]
    
    gp_df = gp.GeoDataFrame(pd_df, geometry='geometry')
    
    assert vrp.get_geoId(101,gp_df) == 456, "incorrect geo Id"

In [39]:
def test_get_geoId_exception():
    """ Testing getting geo id (census block Id) from geo Id files, case where the geo id does not exist in the file
        This case returns -1
    """
    pd_df = pd.DataFrame()
    pd_df['GEOID'] = [456, 457]
    pd_df['CBPZONE'] = [1234, 1235]
    pd_df['MESOZONE'] = [101,1001]
    pd_df['geometry'] = [Point(-101.91, 33.55), Point(-98.53, 29.53)]
    
    gp_df = gp.GeoDataFrame(pd_df, geometry='geometry')
    
    assert vrp.get_geoId(2003,gp_df) == -1, "incorrect geo Id"

In [8]:
def test_create_data_model_delivery_external_normal():
    df_prob = pd.read_csv('test_data/df_prob_delivery.csv')
    c_prob = pd.read_csv('test_data/c_prob_delivery.csv')
    v_df = pd.read_csv('test_data/v_df_delivery.csv')
    vc_prob = pd.read_csv('test_data/vc_prob_delivery.csv')
    CBGzone_df = pd.read_csv('test_data/CBGzone_df.csv')
    tt_df = pd.read_csv('test_data/sel_tt_delivery.csv')
    dist_df = pd.read_csv('test_data/sel_dist_delivery.csv')
    prob_type = 'delivery'
    carr_id = 'B2C_2879885.0'
    depot_loc = c_prob.loc[c_prob['carrier_id'] == carr_id]['depot_zone'].values[0]
    veh = 'md_D_Diesel'
    comm = 5
    index = 'external'
    
    data = {}
    with open('test_data/b2c_delivery_external.pickle', 'rb') as handle:
        data = pickle.load(handle)
    
    ret_data = vrp.create_data_model(df_prob, depot_loc, prob_type, v_df, vc_prob, c_prob, carr_id,
                                                CBGzone_df, tt_df, dist_df, veh, comm, index)
    
    TestCase().assertDictEqual(ret_data, data, "incorrect data dictionary created")
    

In [9]:
def test_create_data_model_pickup_delivery_external_normal():
    df_prob = pd.read_csv('test_data/df_prob_pickup_delivery.csv')
    c_prob = pd.read_csv('test_data/c_prob_pickup_delivery.csv')
    v_df = pd.read_csv('test_data/v_df_pickup_delivery.csv')
    vc_prob = pd.read_csv('test_data/vc_prob_pickup_delivery.csv')
    CBGzone_df = pd.read_csv('test_data/CBGzone_df.csv')
    tt_df = pd.read_csv('test_data/sel_tt_pickup_delivery.csv')
    dist_df = pd.read_csv('test_data/sel_dist_pickup_delivery.csv')
    prob_type = 'pickup_delivery'
    carr_id = 'B2B_2627740_0hdt_D'
    depot_loc = c_prob.loc[c_prob['carrier_id'] == carr_id]['depot_zone'].values[0]
    veh = 'hdt_D_Diesel'
    comm = 2
    index = 'external'
    
    data = {}
    with open('test_data/b2b_pickup_delivery_external.pickle', 'rb') as handle:
        data = pickle.load(handle)
    
    ret_data = vrp.create_data_model(df_prob, depot_loc, prob_type, v_df, vc_prob, c_prob, carr_id,
                                                CBGzone_df, tt_df, dist_df, veh, comm, index)
    
    TestCase().assertDictEqual(ret_data, data, "incorrect data dictionary created")

In [10]:
test_create_data_model_pickup_delivery_external_normal()

picked data:
veh_capacity:  36000  num_veh:  8
